# Titanic Survival Prediction

> ❗ Este modelo é fortemente inspirado [neste post da documentação do Danfo.](https://danfo.jsdata.org/examples/titanic-survival-prediction-using-danfo.js-and-tensorflow.js)
> 

### Importar *libs*

In [114]:
import dfd, { DataFrame } from 'danfojs-node';

const tf = dfd.tensorflow;

### Carregar dados

Os dados serão carregados de um `csv`, pré-baixado no diretório `./data` e que foi [obtido da Universidade de Stanford](https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv).

In [115]:


const load_data = async () => await dfd.readCSV( "./data/titanic.csv" );

let df: DataFrame = await load_data();

df.head().print(); // printa as primeiras 5 linhas do dataframe

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Name              │ Sex               │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 0          │ 0                 │ 3                 │ Mr. Owen Harris…  │ male              │ 22                │ 1                 │ 0                 │ 7.25              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 1          │ 1                 │ 1                 │ Mrs. John Bradl…  │ female            │ 38                │ 1         

### Investigar os dados

Vamos operar alguns comandos para entender melhor os dados que temos em mãos.

In [116]:
df.ctypes.print(); // printa os tipos de dados de cada coluna

╔═════════════════════════╤═════════╗
║ Survived                │ int32   ║
╟─────────────────────────┼─────────╢
║ Pclass                  │ int32   ║
╟─────────────────────────┼─────────╢
║ Name                    │ string  ║
╟─────────────────────────┼─────────╢
║ Sex                     │ string  ║
╟─────────────────────────┼─────────╢
║ Age                     │ int32   ║
╟─────────────────────────┼─────────╢
║ Siblings/Spouses Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Parents/Children Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Fare                    │ float32 ║
╚═════════════════════════╧═════════╝



Vemos que temos duas colunas que são do tipo `string` ('Name' e 'Sex'). 

Para nosso modelo, precisamos que todas as colunas sejam numéricas. As redes neurais que vamos utilizar não conseguem lidar com strings muito bem.

Mas antes de resolver esse problema, vamos ter um panorama estatístico dos dados.

In [117]:
df.describe().print(); // printa estatísticas descritivas do dataframe

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ count      │ 887               │ 887               │ 887               │ 887               │ 887               │ 887               ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ mean       │ 0.3855693348365…  │ 2.3055242390078…  │ 29.471443066516…  │ 0.5253664036076…  │ 0.3833145434047…  │ 32.305420180383…  ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ std        │ 0.4870041177510…  │ 0.8366620036697…  │ 

### Limpeza de dados

Essa coluna 'Name' não é relevante para nosso modelo, então vamos removê-la.

In [118]:
df = df.drop( { columns: [ 'Name' ] } ); // remove a coluna "Name" do dataframe
df.head().print();

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Sex               │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 0          │ 0                 │ 3                 │ male              │ 22                │ 1                 │ 0                 │ 7.25              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 1          │ 1                 │ 1                 │ female            │ 38                │ 1                 │ 0                 │ 71.2833           ║
╟────────────┼───────────────────┼───────────────────┼────────────────

Agora vamos atacar a coluna 'Sex'. Vamos transformar essa coluna em uma coluna numérica.

Vamos primeiro investigar que valores ela assume.

In [119]:
const uniqueValues = df[ 'Sex' ].unique();
uniqueValues.print();

╔═══╤════════╗
║ 0 │ male   ║
╟───┼────────╢
║ 1 │ female ║
╚═══╧════════╝



Vamos verificar se há valores nulos na amostra.

In [120]:
const isNA = df.isNa(); // verifica se há valores NA, associando true a valores NA e false a valores não NA
const isThereNA = isNA.values.some( ( value ) => value === true ); // verifica se há algum valor NA
console.log( isThereNA );

false


Legal. Agora sabemos que não há valores inválidos em toda a amostra e que a coluna 'Sex' só assume os valores 'male' ou 'female'.

Então vamos transformar esses valores em 0 (male) e 1 (female).

In [121]:
df[ 'Sex' ] = df[ 'Sex' ].map( value => value === 'male' ? 0 : 1 );
df.head().print();

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Sex               │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 0          │ 0                 │ 3                 │ 0                 │ 22                │ 1                 │ 0                 │ 7.25              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ 1          │ 1                 │ 1                 │ 1                 │ 38                │ 1                 │ 0                 │ 71.2833           ║
╟────────────┼───────────────────┼───────────────────┼────────────────

Vamos checar novamente os tipos das colunas.

In [122]:
df.ctypes.print();

╔═════════════════════════╤═════════╗
║ Survived                │ int32   ║
╟─────────────────────────┼─────────╢
║ Pclass                  │ int32   ║
╟─────────────────────────┼─────────╢
║ Sex                     │ int32   ║
╟─────────────────────────┼─────────╢
║ Age                     │ int32   ║
╟─────────────────────────┼─────────╢
║ Siblings/Spouses Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Parents/Children Aboard │ int32   ║
╟─────────────────────────┼─────────╢
║ Fare                    │ float32 ║
╚═════════════════════════╧═════════╝



Legal. Apenas colunas numéricas. Podemos prosseguir com nossa análise.

### Ajustes na amostra

Se queremos prever a change de um adulto morrer caso estivesse no Titanic, precisamos remover os dados de crianças para adequar a amostra. 

In [123]:
const MAIORIDADE = 18;

let dfMaioridade = df.query( df[ 'Age' ].ge( MAIORIDADE ) ); // aplica um filtro e cria um novo dataframe com os dados filtrados

dfMaioridade.describe().print(); // verifica se o mínimo da idade ficou em 18

╔════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╤═══════════════════╗
║            │ Survived          │ Pclass            │ Sex               │ Age               │ Siblings/Spouse…  │ Parents/Childre…  │ Fare              ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ count      │ 757               │ 757               │ 757               │ 757               │ 757               │ 757               │ 757               ║
╟────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────┼───────────────────╢
║ mean       │ 0.3659180977542…  │ 2.2483487450462…  │ 0.3315719947159…  │ 32.953104359313…  │ 0.3315719947159…  │ 0.2694848084544…  │ 32.519670673711…  ║
╟────────────┼───────────────────┼───────────────────┼────────────────

Perfeito. Agora temos somente dados de pessoas maiores de idade.


### Modelo

Agora que temos os dados limpos, vamos avançar para a criação do modelo.

A primeira coisa que devemos fazer é criar uma função que normalize nossos dados e retorne tensores.

In [124]:
const processData = (df) => {
  let x_train, y_train;
    x_train = df.iloc({ columns: [`1:`] })
    y_train = df['Survived']

    // padroniza os dados usando MinMaxScaler
    let scaler = new dfd.MinMaxScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    return [x_train.tensor, y_train.tensor] //retorna os dados como tensores
}

const tensors = processData(dfMaioridade)

Agora podemos criar uma função que retorna nosso modelo.
Nessa iteração nosso modelo de rede neural sequencial com 4 camadas densas, sendo que a função de ativação da última camada é a função sigmóide.

In [125]:
function get_model() {
  const model = tf.sequential();
  model.add(tf.layers.dense({ inputShape: [6], units: 124, activation: 'relu', kernelInitializer: 'leCunNormal' }));
  model.add(tf.layers.dense({ units: 64, activation: 'relu' }));
  model.add(tf.layers.dense({ units: 32, activation: 'relu' }));
  model.add(tf.layers.dense({ units: 1, activation: "sigmoid" }))
  model.summary();
  return model
}

E agora que temos o modelo, podemos treiná-lo. A função train irá treinar o modelo e retornar um objeto com as métricas de treinamento.

In [126]:
async function train(data) {
  const model = get_model()
  const x_train = data[0]
  const y_train = data[1]

  model.compile({
      optimizer: "rmsprop",
      loss: 'binaryCrossentropy',
      metrics: ['accuracy'],
  });

  console.log("Treinamento iniciado....")
  await model.fit(x_train, y_train,{
      batchSize: 32,
      epochs: 15,
      validationSplit: 0.2,
      callbacks:{
          onEpochEnd: async(epoch, logs)=>{
              console.log(`EPOCH (${epoch + 1}): Acurácia do treinamento: ${(logs.acc * 100).toFixed(2)} |
                                                   Acurácia da Validação:  ${(logs.val_acc * 100).toFixed(2)} | Validation Loss: ${(logs.val_loss * 100).toFixed(2)}`);
          }
      }
  });

  return model
}

const trainedModel = await train(tensors)

__________________________________________________________________________________________
Layer (type)                Input Shape               Output shape              Param #   
dense_Dense33 (Dense)       [[null,6]]                [null,124]                868       
__________________________________________________________________________________________
dense_Dense34 (Dense)       [[null,124]]              [null,64]                 8000      
__________________________________________________________________________________________
dense_Dense35 (Dense)       [[null,64]]               [null,32]                 2080      
__________________________________________________________________________________________
dense_Dense36 (Dense)       [[null,32]]               [null,1]                  33        
Total params: 10981
Trainable params: 10981
Non-trainable params: 0
__________________________________________________________________________________________
Treinamento iniciado..

No output acima é possível verificar a acurácia do modelo, tanto em treinamento quanto em validação e a perda em validação.

#### Próximos passos para você

- Melhorar a acurácia do modelo
- Salvar o modelo treinado (model.save())
- Criar uma pequena aplicação web para prever a chance de sobrevivência de um adulto no Titanic usando Deno Fresh e Tensorflow.JS ([Veja este exemplo](https://paul.kinlan.me/ml-deno-fresh-tensorflow/))